In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import re
import os
import random
import torch
import easydict
print(torch.cuda.is_available(), torch.cuda.device_count())

os.chdir('./code/')
print(os.getcwd())

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from optuna.pruners import ThresholdPruner

import wandb
wandb.login(key = '95d4fb2d02fe3866b9b32239df088d24cccd2efe')

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings(action = 'ignore')

from src.utils import Logger, Setting, rmse, models_load
from src.train.trainer import train, valid, test, RMSELoss

from src.data import context_data_load, context_data_split, context_data_loader # 'FM', 'FFM'에서 사용되는 데이터 처리 모듈
from src.data import dl_data_load, dl_data_split, dl_data_loader                # 'NCF', 'WDN', 'DCN'에서 사용되는 데이터 처리 모듈
from src.data import image_data_load, image_data_split, image_data_loader       # 'CNN_FM'에서 사용되는 데이터 처리 모듈
from src.data import text_data_load, text_data_split, text_data_loader          # 'DeepCoNN'에서 사용되는 데이터 처리 모듈

from src.models.FM.FM_model import FactorizationMachineModel
from src.models.FFM.FFM_model import FieldAwareFactorizationMachineModel
from src.models.WDN.WDN_model import WideAndDeepModel
from src.models.NCF.NCF_model import NeuralCollaborativeFiltering
from src.models.DCN.DCN_model import StackedDeepCrossNetworkModel, ParallelDeepCrossNetworkModel
from src.models.CNN_FM.CNN_FM_model import CNN_FM
from src.models.DeepCoNN.DeepCoNN_model import DeepCoNN

from src.ensembles import ensembles

# Seed 고정
SEED = 42
def seed_everything(seed) :
    random.seed(seed)
    np.random.seed(seed)
    # tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    torch.use_deterministic_algorithms(True)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(SEED)

True 1
/data/ephemeral/Project/code


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jw0112. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


In [2]:
args = easydict.EasyDict({
    'data_path' : './data/',  # Data path 설정
    'saved_pred_path' : './submit',
    'saved_model_path' : './saved_models',  # Saved Model path 설정
    'model' : None,  # 학습 및 예측할 모델 선택 (None으로 초기화, 사용 전에 설정 필요)
    'data_shuffle' : True,  # 데이터 셔플 여부 조정
    'test_size' : 0.1,  # Train/Valid split 비율 조정
    'seed' : 42,  # Seed 값 조정
    'use_best_model' : True,  # 검증 성능이 가장 좋은 모델 사용 여부 설정
    'patience' : 10, # Earlystop의 patience를 조정
    'runname' : 'CNN_FM_HPO',

    # TRAINING OPTION
    'batch_size' : 256,  # Batch size 조정
    'epochs' : 30,  # Epoch 수 조정
    'lr' : 1e-3,  # Learning Rate 조정
    'loss_fn' : 'RMSE',  # 손실 함수 변경 (MSE 또는 RMSE)
    'optimizer' : 'NAdam',  # 최적화 함수 변경 (SGD, Adam, AdamW, NAdam, SparseAdam)
    'weight_decay' : 1e-6,  # Adam optimizer에서 정규화에 사용하는 값 조정
    'lr_scheduler' : None, # Learning Rate Scheduler 설정

    # GPU
    'device' : 'cuda',  # 학습에 사용할 Device 조정

    # FM, FFM, NCF, WDN, DCN Common OPTION
    'embed_dim' : 16,  # FM, FFM, NCF, WDN, DCN에서 embedding시킬 차원 조정
    'dropout' : 0.2,  # NCF, WDN, DCN에서 Dropout rate 조정
    'mlp_dims' : (16, 16),  # NCF, WDN, DCN에서 MLP Network의 차원 조정

    # DCN
    'num_layers' : 5,  # Cross Network의 레이어 수 조정

    # CNN_FM
    'cnn_embed_dim' : 64,  # CNN_FM에서 user와 item에 대한 embedding시킬 차원 조정
    'cnn_latent_dim' : 12,  # CNN_FM에서 user/item/image에 대한 latent 차원 조정

    # DeepCoNN
    'vector_create' : False,  # DEEP_CONN에서 text vector 생성 여부 조정 (최초 학습에만 True로 설정)
    'deepconn_embed_dim' : 32,  # DEEP_CONN에서 user와 item에 대한 embedding시킬 차원 조정
    'deepconn_latent_dim' : 10,  # DEEP_CONN에서 user/item/image에 대한 latent 차원 조정
    'conv_1d_out_dim' : 50,  # DEEP_CONN에서 1D conv의 출력 크기 조정
    'kernel_size' : 3,  # DEEP_CONN에서 1D conv의 kernel 크기 조정
    'word_dim' : 768,  # DEEP_CONN에서 1D conv의 입력 크기 조정
    'out_dim' : 32  # DEEP_CONN에서 1D conv의 출력 크기 조정
})

## CNN_FM

In [3]:
def objective_CNN_FM(trial : Trial, dataset) :
    
    # Setting for Log
    setting = Setting()
    setting.seed_everything(SEED)

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    # wandb
    wandb.init(project = 'Book_Recommendation')
    wandb.run.name = args.runname + str(trial)
    wandb.run.save()
    wandb.config.update(args)
    
    # Common Parameter
    args.batch_size = trial.suggest_categorical('batch_size', [256, 512, 1024])
    args.lr = trial.suggest_categorical('lr', [1e-3, 0.01, 0.1, 0.5])
    args.optimizer = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW', 'NAdam'])
    args.weight_decay = trial.suggest_categorical('weight_decay', [1e-7, 5e-7, 1e-6, 5e-6, 1e-5])
    # args.lr_scheduler = trial.suggest_categorical('lr_scheduler', ['Exponential', 'CosineAnnealingWarmRestarts'])
    
    # CNN_FM Parameter
    args.cnn_embed_dim = trial.suggest_int('cnn_embed_dim', 16, 64)
    args.cnn_latent_dim = trial.suggest_int('cnn_latent_dim', 8, 16)

    args.model = 'CNN_FM'
    args.loss_fn = 'RMSE'
    args.epochs = 30
    
    data = image_data_split(args, dataset)
    data = image_data_loader(args, dataset)
    
    model = CNN_FM(args, data).to(args.device)
    model, minimum_loss = train(args, model, data, logger, setting)
    return minimum_loss

In [4]:
CNN_FM_Dataset = image_data_load(args)

129777it [01:00, 2159.96it/s]
52000it [00:24, 2135.16it/s]


In [5]:
# Study Optuna
study = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = SEED))
study.optimize(lambda trial : objective_CNN_FM(trial, CNN_FM_Dataset), n_trials = 50, show_progress_bar = True)
print(f'Best trial : score {study.best_trial.value}, \n params = {study.best_trial.params} \n')

[I 2023-12-19 23:31:16,168] A new study created in memory with name: no-name-b1aba7e5-0910-4dc3-a0a8-5c2fe03996bf
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
  3%|▎         | 1/30 [00:15<07:17, 15.10s/it]

Epoch : 1, Train Loss : 2.679, Valid Loss : 2.418, LR : 0.001, EarlyStop Count : 0


  7%|▋         | 2/30 [00:29<06:46, 14.52s/it]

Epoch : 2, Train Loss : 2.381, Valid Loss : 2.366, LR : 0.001, EarlyStop Count : 0


...

Epoch : 6, Train Loss : inf, Valid Loss : inf, LR : 0.5, EarlyStop Count : 6
Pruning.



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,▁▁▁▁▁▁
Best Loss,999999999
Train Loss,inf
Valid Loss,inf


[I 2023-12-20 03:12:26,609] Trial 49 finished with value: 999999999.0 and parameters: {'batch_size': 1024, 'lr': 0.5, 'optimizer': 'SGD', 'weight_decay': 5e-06, 'cnn_embed_dim': 30, 'cnn_latent_dim': 9}. Best is trial 21 with value: 2.1674882570902505.


Best trial : score 2.1674882570902505, 
 params = {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 1e-05, 'cnn_embed_dim': 38, 'cnn_latent_dim': 11} 



In [6]:
print(f'Best trial : score {study.best_trial.value}, \n params = {study.best_trial.params} \n')

Best trial : score 2.1674882570902505, 
 params = {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 1e-05, 'cnn_embed_dim': 38, 'cnn_latent_dim': 11} 



In [8]:
X_data, y_data = CNN_FM_Dataset['img_train'][['user_id', 'isbn', 'img_vector']], CNN_FM_Dataset['img_train']['rating']

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = SEED)
valid_rmse = []
pred_df = pd.DataFrame()

# Stratified K-Fold 사용하여 Best Parameter 학습
for fold, (train_idx, valid_idx) in tqdm.tqdm(enumerate(skf.split(X_data, y_data)), total = skf.n_splits) : 
    
    print(f'--------------------------------------------------- Fold : {fold} ---------------------------------------------------')
    # Train Set과 Valid Set 분할    
    X_train, y_train = X_data.iloc[train_idx], y_data.iloc[train_idx]
    X_valid, y_valid = X_data.iloc[valid_idx], y_data.iloc[valid_idx]
    
    # Best Parameter 학습
    param = study.best_trial.params # {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 1e-05, 'cnn_embed_dim': 38, 'cnn_latent_dim': 11}
    
    # Setting for Log
    setting = Setting()
    setting.seed_everything(SEED)

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    # wandb
    wandb.init(project = 'Book_Recommendation')
    wandb.run.name = 'CNN_FM Best Parameter Fold : ' + str(fold)
    wandb.run.save()
    wandb.config.update(args)
    
    # Common Parameter
    args.batch_size = param['batch_size']
    args.lr = param['lr']
    args.optimizer = param['optimizer']
    args.weight_decay = param['weight_decay']
    
    # CNN_FM Parameter
    args.cnn_embed_dim = param['cnn_embed_dim']
    args.cnn_latent_dim = param['cnn_latent_dim']

    args.model = 'CNN_FM'
    args.loss_fn = 'RMSE'
    args.epochs = 30
    
    data = image_data_split(args, CNN_FM_Dataset)
    data = image_data_loader(args, CNN_FM_Dataset)
    
    model = CNN_FM(args, data).to(args.device)
    model, minimum_loss = train(args, model, data, logger, setting)
    
    valid_rmse.append(minimum_loss)
    print(f'Valid Set {fold + 1} 번째 Fold RMSE : {minimum_loss:.5f}')
    
    # Predict
    pred = test(args, model, data, setting)
    pred_df[f'pred_{fold}'] = pred
    
print(f'CNN_FM RMSE 평균 : {np.array(valid_rmse).mean():.5f} \n')

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------- Fold : 0 ---------------------------------------------------


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,█▅▃▂▁
Train Loss,█▃▂▂▁
Valid Loss,█▅▃▂▁
Best Loss,2.19394
Train Loss,2.06341
Valid Loss,2.19394


Epoch : 1, Train Loss : 2.466, Valid Loss : 2.310, LR : 0.01, EarlyStop Count : 0


...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,██▇▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁
Valid Loss,▇█▆▃▄▅▃▃▁▂▃▅▂▁▁▁▃▁▂▂▂▂▃▂
Best Loss,2.16749
Train Loss,1.68109
Valid Loss,2.19941


Valid Set 10 번째 Fold RMSE : 2.16749


100%|██████████| 10/10 [58:36<00:00, 351.61s/it]

CNN_FM RMSE 평균 : 2.16749 



In [9]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['rating'] = (pred_df['pred_0'] + pred_df['pred_1'] + pred_df['pred_2'] + pred_df['pred_3'] + pred_df['pred_4'] + 
                               pred_df['pred_5'] + pred_df['pred_6'] + pred_df['pred_7'] + pred_df['pred_8'] + pred_df['pred_9']) / 10
submit = sample_submission[['user_id', 'isbn', 'rating']]
submit

,user_id,isbn,rating
0,11676,0002005018,6.670147
1,116866,0002005018,7.761848
2,152827,0060973129,7.723060
3,157969,0374157065,6.860052
4,67958,0399135782,8.050864
...,...,...,...
76694,278543,1576734218,6.883442
76695,278563,3492223710,6.968682
76696,278633,1896095186,7.024070
76697,278668,8408044079,5.672930


In [10]:
submit.to_csv(f'./submit/CNN_FM_HPO_{str(str(study.best_trial.value).split(".")[1][:5])}.csv', index = False)

### CNN_FM HPO Result
- best_params : {'batch_size': 512, 'lr': 0.001, 'optimizer': 'SGD', 'weight_decay': 1e-07, 'cnn_embed_dim': 50, 'cnn_latent_dim': 37}
    - LB : 2.1889 / CV : 2.180

- best_params : {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 5e-06, 'cnn_embed_dim': 36, 'cnn_latent_dim': 14}
    - LB : 2.1766 / CV : 2.174

- best_params : {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 1e-05, 'cnn_embed_dim': 38, 'cnn_latent_dim': 11}
    - LB : 2.1739 / CV : 2.167

## DeepCoNN

In [6]:
def objective_DeepCoNN(trial, dataset) :
    
    # Setting for Log
    setting = Setting()
    setting.seed_everything(SEED)

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    args.runname = 'DeepCoNN_HPO'
    
    # wandb
    wandb.init(project = 'Book_Recommendation')
    wandb.run.name = args.runname + str(trial)
    wandb.run.save()
    wandb.config.update(args)
    
    # Common Parameter
    args.batch_size = trial.suggest_categorical('batch_size', [256, 512, 1024])
    args.lr = trial.suggest_categorical('lr', [1e-3, 0.01, 0.1, 0.5])
    args.optimizer = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW', 'NAdam'])
    args.weight_decay = trial.suggest_categorical('weight_decay', [1e-7, 5e-7, 1e-6, 5e-6, 1e-5])
    
    # DeepCoNN Parameter
    args.deepconn_embed_dim = trial.suggest_int('deepconn_embed_dim', 16, 64)   # DEEP_CONN에서 user와 item에 대한 embedding시킬 차원 조정
    args.deepconn_latent_dim = trial.suggest_int('deepconn_latent_dim', 4, 16)  # DEEP_CONN에서 user/item/image에 대한 latent 차원 조정
    args.conv_1d_out_dim = trial.suggest_int('conv_1d_out_dim', 32, 128)        # DEEP_CONN에서 1D conv의 출력 크기 조정
    args.kernel_size = 3 # trial.suggest_int('kernel_size', 1, 3)               # DEEP_CONN에서 1D conv의 kernel 크기 조정
    args.word_dim = 768 # trial.suggest_int('word_dim', 256, 2048)              # DEEP_CONN에서 1D conv의 입력 크기 조정
    args.out_dim = trial.suggest_int('out_dim', 16, 64)                         # DEEP_CONN에서 1D conv의 출력 크기 조정
    
    args.model = 'DeepCoNN'
    args.loss_fn = 'RMSE'
    args.epochs = 30
    
    data = text_data_split(args, dataset)
    data = text_data_loader(args, dataset)
    
    model = DeepCoNN(args, data).to(args.device)
    model, minimum_loss = train(args, model, data, logger, setting)
    
    return minimum_loss

In [3]:
DeepCoNN_Dataset = text_data_load(args)

Check Vectorizer
Vector Load
Check Vectorizer
Vector Load


In [7]:
# Study Optuna
study = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = SEED))
study.optimize(lambda trial : objective_DeepCoNN(trial, DeepCoNN_Dataset), n_trials = 50, show_progress_bar = True)
print(f'Best trial : score {study.best_trial.value}, \n params = {study.best_trial.params} \n')

[I 2023-12-19 12:36:08,581] A new study created in memory with name: no-name-c914aa87-602f-44dc-baad-ac102cfd7d76


  0%|          | 0/50 [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,▁▁▁
Best Loss,999999999
Train Loss,nan
Valid Loss,nan


Epoch : 1, Train Loss : 5.039, Valid Loss : 3.535, LR : 0.001, EarlyStop Count : 0


...

 33%|███▎      | 10/30 [03:47<07:35, 22.79s/it]

Epoch : 11, Train Loss : 1.403, Valid Loss : 2.378, LR : 0.1, EarlyStop Count : 10
Early Stopping. Best Valid_loss : 2.226



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▁▁▁▁▁▁▁▁▁▁
Valid Loss,▁▂▄▅▆▆▆▇▇██
Best Loss,2.22577
Train Loss,1.40264
Valid Loss,2.37847


[I 2023-12-19 15:33:45,001] Trial 49 finished with value: 2.2257671117782594 and parameters: {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 17, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 77, 'out_dim': 31}. Best is trial 42 with value: 2.216065756479899.
Best trial : score 2.216065756479899, 
 params = {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 18, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 52, 'out_dim': 38} 



In [15]:
print(f'Best trial : score {study.best_trial.value}, \n params = {study.best_trial.params} \n')

Best trial : score 2.216065756479899, 
 params = {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 18, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 52, 'out_dim': 38} 



In [11]:
X_data, y_data = DeepCoNN_Dataset['text_train'][['user_id', 'isbn', 'user_summary_merge_vector', 'item_summary_vector']], DeepCoNN_Dataset['text_train']['rating']

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = SEED)
valid_rmse = []
pred_df = pd.DataFrame()

# Stratified K-Fold 사용하여 Best Parameter 학습
for fold, (train_idx, valid_idx) in tqdm.tqdm(enumerate(skf.split(X_data, y_data)), total = skf.n_splits) : 
    
    print(f'--------------------------------------------------- Fold : {fold} ---------------------------------------------------')
    # Train Set과 Valid Set 분할    
    X_train, y_train = X_data.iloc[train_idx], y_data.iloc[train_idx]
    X_valid, y_valid = X_data.iloc[valid_idx], y_data.iloc[valid_idx]
    
    # Best Parameter 학습
    param = study.best_params # {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 18, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 52, 'out_dim': 38}
    
    # Setting for Log
    setting = Setting()
    setting.seed_everything(SEED)

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    # wandb
    wandb.init(project = 'Book_Recommendation')
    wandb.run.name = 'CNN_FM Best Parameter Fold : ' + str(fold)
    wandb.run.save()
    wandb.config.update(args)
    
    # Common Parameter
    args.batch_size = param['batch_size']
    args.lr = param['lr']
    args.optimizer = param['optimizer']
    args.weight_decay = param['weight_decay']
    
    # DeepCoNN Parameter
    args.deepconn_embed_dim = param['deepconn_embed_dim']
    args.deepconn_latent_dim = param['deepconn_latent_dim']
    args.conv_1d_out_dim = param['conv_1d_out_dim']
    args.kernel_size = 3
    args.word_dim = 768
    args.out_dim = param['out_dim']
    
    args.model = 'DeepCoNN'
    args.loss_fn = 'RMSE'
    args.epochs = 30
    
    data = text_data_split(args, DeepCoNN_Dataset)
    data = text_data_loader(args, DeepCoNN_Dataset)
    
    model = DeepCoNN(args, data).to(args.device)
    model, minimum_loss = train(args, model, data, logger, setting)
    
    valid_rmse.append(minimum_loss)
    print(f'Valid Set {fold + 1} 번째 Fold RMSE : {minimum_loss:.5f}')
    
    # Predict
    pred = test(args, model, data, setting)
    pred_df[f'pred_{fold}'] = pred
    
print(f'CNN_FM RMSE 평균 : {np.array(valid_rmse).mean():.5f} \n')

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------- Fold : 0 ---------------------------------------------------


Epoch : 1, Train Loss : 8.959, Valid Loss : 2.216, LR : 0.1, EarlyStop Count : 0


Epoch : 2, Train Loss : 2.001, Valid Loss : 2.248, LR : 0.1, EarlyStop Count : 1


Epoch : 3, Train Loss : 1.867, Valid Loss : 2.284, LR : 0.1, EarlyStop Count : 2


...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Best Loss,▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▂▁▁▁▁▁▁▁▁▁
Valid Loss,▁▂▃▄▆▆▆▇▆█▇
Best Loss,2.21607
Train Loss,1.37997
Valid Loss,2.38493


Valid Set 10 번째 Fold RMSE : 2.21607


100%|██████████| 10/10 [42:44<00:00, 256.49s/it]

CNN_FM RMSE 평균 : 2.21607 



In [12]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['rating'] = (pred_df['pred_0'] + pred_df['pred_1'] + pred_df['pred_2'] + pred_df['pred_3'] + pred_df['pred_4'] + 
                               pred_df['pred_5'] + pred_df['pred_6'] + pred_df['pred_7'] + pred_df['pred_8'] + pred_df['pred_9']) / 10
submit = sample_submission[['user_id', 'isbn', 'rating']]
submit

,user_id,isbn,rating
0,11676,0002005018,6.728746
1,116866,0002005018,7.908971
2,152827,0060973129,8.257817
3,157969,0374157065,6.811160
4,67958,0399135782,7.942155
...,...,...,...
76694,278543,1576734218,7.139505
76695,278563,3492223710,5.901618
76696,278633,1896095186,7.014034
76697,278668,8408044079,5.049523


In [17]:
submit.to_csv(f'./submit/DeepCoNN_HPO_{str(str(study.best_trial.value).split(".")[1][:5])}.csv', index = False)

### DeepCoNN HPO Result
- best_params : {'batch_size': 256, 'lr': 0.01, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 53, 'deepconn_latent_dim': 11, 'conv_1d_out_dim': 106}
    - LB : 2.2438 / CV : 2.23975

- best_params : {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 18, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 52, 'out_dim': 38} 
    - LB : 2.2211 / CV : 2.21607

## Ensemble

In [51]:
!python ensemble.py --ensemble_files 'Catboost_1250,CNN_FM_HPO_16748,DeepCoNN_HPO_21606' --ensemble_strategy 'weighted' --ensemble_weight '0.75,0.18,0.07'

#### Catboost
- best_params : 
    - LB : 2.1250 / CV : ___


#### CNN_FM
- best_params : {'batch_size': 1024, 'lr': 0.01, 'optimizer': 'SGD', 'weight_decay': 1e-05, 'cnn_embed_dim': 38, 'cnn_latent_dim': 11}
    - LB : 2.1739 / CV : 2.167

#### DeepCoNN
- best_params : {'batch_size': 512, 'lr': 0.1, 'optimizer': 'NAdam', 'weight_decay': 5e-07, 'deepconn_embed_dim': 18, 'deepconn_latent_dim': 4, 'conv_1d_out_dim': 52, 'out_dim': 38} 
    - LB : 2.2211 / CV : 2.21607

#### Weight
- Catboost : CNN_FM : DeepCoNN = 0.7 : 0.2 : 0.1 => **LB : 2.1207**
- Catboost : CNN_FM : DeepCoNN = 0.75 : 0.18 : 0.07 => **LB : 2.1202**